In [9]:
import pandas as pd
import json

# INPUT_FILE = './data/split_54_20250614.csv'
INPUT_FILE = './gt_n5_20250618_divide_5_new_deduplicated.csv'
# OUTPUT_FILE = './data/llada_format/split_54_0614_tojson_eng.json'
OUTPUT_FILE = './data/llada_format/gt_n5_20250618_new_tojson_eng.json'



df = pd.read_csv(INPUT_FILE)

data_list = []

for idx, row in df.iterrows():
    order = row['order']  # 문자열
    machine_raw = row['machine']
    gt_raw = row['gt']

    # 1) order에서 날짜, 품목, 수량 등 추출
    order_parts = order.split()
    date = order_parts[1]
    item = order_parts[2]
    cost = order_parts[3]
    urgent = order_parts[4]
    qty = order_parts[5]

    # 2) machine, gt 멀티라인 텍스트 → 리스트 파싱
    machines = []
    for line in machine_raw.split('\n'):
        if line.strip():
            # '• machine24 5.0' → ['machine24', '5.0']
            parts = line.strip('• ').split()
            machines.append((parts[0], parts[1]))

    gts = []
    for line in str(gt_raw).split('\n'):
        line = line.strip()
        
        if not line:
            gts.append(('nan', 'nan'))  # 의도적으로 빈 줄도 처리
            continue
        
        if line.lower() == 'nan':
            gts.append(('nan', 'nan'))
            continue
        
        parts = line.lstrip('• ').split()
        if len(parts) >= 2:
            gts.append((parts[0], parts[1]))
        else:
            gts.append(('nan', 'nan'))  # 불완전한 정보도 의도적 처리 가능


    # 3) 자연어 문장 생성
    input_text = f"Delivery date is {date}, item is {item}, quantity is {qty}, cost per unit is {cost}, urgent quantity is {urgent}. Available machines and processable quantities are"
    input_text += ", ".join([f"{m[0]}: {m[1]}" for m in machines]) + "."

    output_text = "Assigned machines are " + ", ".join([f"{g[0]}: {g[1]}" for g in gts]) + "."

    data_list.append({"input": input_text, "output": output_text})

# 4) JSON 저장
with open(OUTPUT_FILE, 'w', encoding='utf-8') as f:
    json.dump(data_list, f, ensure_ascii=False, indent=2)

In [3]:
# test json 파일 생성 셀

import pandas as pd
import json
from collections import defaultdict

# 1. 입력 파일 읽기
try:
    df = pd.read_csv('./data/ori_divide5_final.csv')
except FileNotFoundError:
    print("오류: './data/aug10_test_final.csv' 파일을 찾을 수 없습니다.")
    print("파일 경로를 확인해주세요.")
    # 파일이 없으면 더 이상 진행하지 않음
    # ipynb 환경이므로 빈 데이터프레임을 만들어 아래 코드 실행 시 에러가 나지 않도록 함
    df = pd.DataFrame(columns=['order', 'machine', 'gt'])

data_list = []

# 2. 데이터프레임의 각 행을 순회하며 데이터 변환
# enumerate를 사용하여 행 번호(인덱스)를 함께 가져옵니다.
for idx, row in df.iterrows():
    # 3. 날짜 생성 (1번 행 -> date401, 2번 행 -> date402, ...)
    # idx는 0부터 시작하므로 +401을 해줍니다.
    current_date = f"date{idx + 401}"

    # 4. input 텍스트 생성
    order_raw = row['order']
    input_sentences = []
    
    # order 컬럼이 비어있지 않고, 문자열일 경우에만 처리
    if pd.notna(order_raw) and isinstance(order_raw, str):
        order_lines = order_raw.strip().split('\n')
        for i, line in enumerate(order_lines):
            line = line.strip()
            if not line:
                continue
            
            # '•' 문자 제거 후 공백으로 분리
            parts = line.lstrip('• ').split()
            
            # 파싱된 데이터가 최소 4개(기존날짜, 품목, 가격, 긴급, 수량 이므로 5개여야함) 이상인지 확인
            if len(parts) >= 5:
                # order_parts = [기존날짜, 품목, 가격, 긴급, 수량]
                item = parts[1]
                cost = parts[2]
                urgent = parts[3]
                qty = parts[4]

                if i == 0:
                    # 첫 번째 주문 문장
                    sentence = f"Delivery date is {current_date}, item is {item}, quantity is {qty}, cost per unit is {cost}, urgent quantity is {urgent}."
                else:
                    # 두 번째 주문부터의 문장
                    sentence = f" item is {item}, quantity is {qty}, cost per unit is {cost}, urgent quantity is {urgent}."
                input_sentences.append(sentence)

    # 생성된 문장들을 하나로 합침
    input_text = "".join(input_sentences)

    # 5. output 텍스트 생성
    gt_raw = row['gt']
    gt_groups = defaultdict(list)
    
    # gt 컬럼이 비어있지 않고, 문자열일 경우에만 처리
    if pd.notna(gt_raw) and isinstance(gt_raw, str):
        gt_lines = gt_raw.strip().split('\n')
        for line in gt_lines:
            line = line.strip()
            if not line:
                continue
                
            # '•' 문자 제거 후 공백으로 분리
            parts = line.lstrip('• ').split()
            
            # 파싱된 데이터가 최소 4개(품목, 기계, 기존날짜, 수량) 이상인지 확인
            if len(parts) >= 4:
                # gt_parts = [품목, 기계, 기존날짜, 수량]
                item = parts[0]
                machine = parts[1]
                qty = parts[3]
                gt_groups[item].append(f"{machine}: {qty}")

    output_sentences = []
    for item, assignments in gt_groups.items():
        assignments_str = ", ".join(assignments)
        sentence = f"For {item} Assigned machines are, {assignments_str}."
        output_sentences.append(sentence)
        
    # 생성된 문장들을 공백 한 칸으로 연결하여 합침
    output_text = " ".join(output_sentences)

    # 6. 최종 데이터 리스트에 추가
    # input_text나 output_text가 비어있지 않은 경우에만 추가 (의미 없는 데이터 방지)
    if input_text and output_text:
        data_list.append({"input": input_text, "output": output_text})

# 7. JSON 파일로 저장
# 디렉터리가 존재하지 않을 경우를 대비하여 생성하는 로직을 추가할 수 있습니다.
# 여기서는 './data/' 디렉터리가 이미 존재한다고 가정합니다.
output_file_path = './data/test128_divide5_0618_final-Copy.json'
with open(output_file_path, 'w', encoding='utf-8') as f:
    json.dump(data_list, f, ensure_ascii=False, indent=2)

print(f"JSON 파일 생성이 완료되었습니다. 총 {len(data_list)}개의 데이터가 생성되었습니다.")
print(f"파일 위치: {output_file_path}")

JSON 파일 생성이 완료되었습니다. 총 26개의 데이터가 생성되었습니다.
파일 위치: ./data/test128_divide5_0618_final-Copy.json


In [5]:
import json
import pandas as pd
from transformers import AutoTokenizer
import numpy as np

# --- 1. 설정 ---
# 분석할 JSON 파일 경로
json_file_path = './data/test128_divide5_0618_final.json'
# 사용할 토크나이저 모델 이름
model_name = "GSAI-ML/LLaDA-8B-Instruct"

print(f"'{model_name}' 토크나이저를 로드합니다...")

# --- 2. 토크나이저 로드 ---
try:
    # 이전 셀에서 이미 토크나이저가 로드되었을 수 있지만,
    # 이 셀만 독립적으로 실행할 수 있도록 다시 로드합니다.
    tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
except Exception as e:
    print(f"토크나이저 로드 중 오류가 발생했습니다: {e}")
    print("이전 셀에서 'transformers' 라이브러리가 올바르게 설치되고 모델이 다운로드되었는지 확인해주세요.")
    # 오류 발생 시 진행을 멈춤
    tokenizer = None

if tokenizer:
    # --- 3. 프롬프트 형식에 필요한 특수 토큰 정의 ---
    # llada_lora-DAY.ipynb의 형식과 동일하게 정의
    bos = tokenizer.bos_token or "<|startoftext|>"
    eos = tokenizer.eos_token or "<|endoftext|>"
    start_user = "<role>user</role>\n"
    start_assistant = "<role>assistant</role>\n"

    print("토크나이저 로드 완료. 특수 토큰을 정의했습니다.")
    print(f"분석할 파일: '{json_file_path}'")

    # --- 4. JSON 파일 로드 및 토큰 길이 계산 ---
    token_lengths = []
    try:
        with open(json_file_path, 'r', encoding='utf-8') as f:
            data_list = json.load(f)

        print(f"총 {len(data_list)}개의 데이터를 로드했습니다. 토큰 길이 계산을 시작합니다...")

        for item in data_list:
            input_text = item.get('input', '')
            output_text = item.get('output', '')

            # 프롬프트 형식으로 합친 텍스트
            combined_text = f"{bos}{start_user}{input_text}\n{start_assistant}{output_text}{eos}"

            # 각 부분 토큰화 및 길이 계산
            input_len = len(tokenizer.encode(input_text))
            output_len = len(tokenizer.encode(output_text))
            combined_len = len(tokenizer.encode(combined_text))

            token_lengths.append({
                'input_length': input_len,
                'output_length': output_len,
                'combined_length': combined_len
            })
        
        print("토큰 길이 계산 완료.")

        # --- 5. Pandas 데이터프레임으로 변환 및 통계 출력 ---
        if token_lengths:
            df_lengths = pd.DataFrame(token_lengths)
            
            print("\n--- 토큰 길이 통계 ---")
            # describe()를 사용하여 통계 정보 출력
            # 더 잘 보이도록 포맷팅 설정
            pd.set_option('display.float_format', lambda x: '%.2f' % x)
            print(df_lengths.describe())
        else:
            print("분석할 데이터가 없습니다.")

    except FileNotFoundError:
        print(f"오류: '{json_file_path}' 파일을 찾을 수 없습니다.")
        print("이전 단계에서 JSON 파일이 정상적으로 생성되었는지, 파일 경로가 올바른지 확인해주세요.")
    except Exception as e:
        print(f"파일 처리 또는 토큰화 중 오류가 발생했습니다: {e}")

'GSAI-ML/LLaDA-8B-Instruct' 토크나이저를 로드합니다...
토크나이저 로드 완료. 특수 토큰을 정의했습니다.
분석할 파일: './data/test128_divide5_0618_final.json'
총 26개의 데이터를 로드했습니다. 토큰 길이 계산을 시작합니다...
토큰 길이 계산 완료.

--- 토큰 길이 통계 ---
       input_length  output_length  combined_length
count         26.00          26.00            26.00
mean         241.00         327.58           580.58
std           36.82          98.23           114.09
min          115.00         134.00           328.00
25%          254.00         255.25           523.00
50%          255.00         348.00           599.00
75%          256.00         402.50           662.50
max          258.00         504.00           772.00
